In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# docs to index
urls = [
    "https://www.langchain.com/docs/get-started/introduction",
    "https://www.langchain.com/docs/get-started/quickstart"
    
]

# Load
docs = [WebBaseLoader(url).load() for url in urls]
doc_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

doc_splits = text_splitter.split_documents(doc_list)

# Add to vector store
vectorstore = FAISS.from_documents(doc_splits, embeddings)

retriever = vectorstore.as_retriever()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
###Router

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field

# Data Model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource"""
    datasource: Literal["vectorstore", "web_search"] = Field(
        description="Given a user query, choose to route it to web search or a vector store.",
        )

# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vector store for questions on these topics. Otherwise, use web-search.
"""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)


question_router = route_prompt | structured_llm_router

print(question_router.invoke({"question": "Who won cricket world cup 2023"}))



datasource='web_search'


In [9]:
### Retrieval Grader

# Data Model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents"""
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'",
    )

# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
grade_llm = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the docs contain keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: {documents} \n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | grade_llm
question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content

print(retrieval_grader.invoke({"question": question, "documents": doc_txt}))



binary_score='no'


In [10]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

#LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#Post-Processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm| StrOutputParser()

# Run
generation = rag_chain.invoke({"context": format_docs(docs), "question": "What is LangChain?"})
print(generation)

I don't know.


In [11]:
# Hallucination Grader

# Data Model
class GradeHallucination(BaseModel):
    """Binary score for hallucination check on generated response"""
    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

#LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structure_llm_grader = llm.with_structured_output(GradeHallucination)

# Prompt
system = """You are a grader assessing whether a generated answer is grounded in/ supported by a set of retrievers.
Give a binary score 'yes' or 'no' to indicate whether the answer is grounded in the facts.
"""

hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Generated answer: {generated_answer} \n set of facts: {retrieved_documents}"),
    ]
)

hallucination_grader = hallucination_prompt | structure_llm_grader

hallucination_grader.invoke({"generated_answer": generation, "retrieved_documents": doc_txt})

GradeHallucination(binary_score='no')

In [13]:
## Answer Grader

# Data Model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'",
    )

# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
grade_llm = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether a generated answer addresses a user question.
Give a binary score 'yes' or 'no' to indicate whether the answer addresses the question."""

answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Generated answer: {generated_answer} \n User question: {question}"),
    ]
)

answer_grader = answer_prompt | grade_llm
answer_grader.invoke({"generated_answer": generation, "question": question})



GradeAnswer(binary_score='no')

In [15]:
# Question Re-writer

#LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt
system = """You are a question re-writer that converts an input to a better version that is optimized \n
for vector store retrieval. Look at the input and try to reason about the underlying semantic intent / meaning. """

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", 
        "Here is the initial question: \n\n {question} \n Formulate an improved quesiton"),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'What are the key concepts and techniques related to agent memory in artificial intelligence?'

In [16]:
### search
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=3
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_13216\487354288.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(max_results=3


In [18]:
from typing import Literal
from typing_extensions import Annotated, TypedDict

class GraphState(TypedDict):
    """
    The state of the graph as it is being executed.
    Attributes:
        question: question
        generation: LLM generated answer
        documents: list of documents
    """

    quesiton: str = Field(description="The question to answer")
    generation: str = Field(description="The LLM generated answer")
    documents: Annotated[list[str], "The list of documents retrieved from the vector store"]
    


In [ ]:
### Assignment